## Modo de uso


1. Las clausulas que se quieren agregar a la base de conocimiento, tienen la siguiente forma: kb.agregar_clausula({Literal("Pompeyano", ("Marco",))}). Este ejemplo es para Pompeyano(Marco)
2. Las variables se colocan en orden alfabetico. Es decir, la primera variable que quieras colocar en la base de conocimiento, debes representarla por la letra a, luego la segunda variable con la letra b, y asi sucesivamente. Ejemplo: agregar_clausula({Literal("-Hombre", ("a",)), Literal("-Gobernante", ("b",)), Literal("-IntentaAsesinar", ("a", "b")), Literal("-Leal", ("a", "b"))})
3. Se debe colocar la conjetura en: conjetura = [{Literal("Odia", ("Marco", "Cesar"))}]. Este ejemplo es para Odia(Marco, Cesar)

## Implementacion del Motor de Inferencia

In [ ]:
class Literal:
    def __init__(self, predicado, argumentos):
        self.predicado = predicado
        self.argumentos = tuple(argumentos)

    def __neg__(self):
        if self.predicado.startswith("-"):
            return Literal(self.predicado[1:], self.argumentos)
        return Literal("-" + self.predicado, self.argumentos)

    def __eq__(self, other):
        return self.predicado == other.predicado and self.argumentos == other.argumentos

    def __hash__(self):
        return hash((self.predicado, self.argumentos))

    def __repr__(self):
        return f"{self.predicado}({', '.join(self.argumentos)})"


class Clausula:
    def __init__(self, literales):
        self.literales = frozenset(literales)

    def __or__(self, other):
        return Clausula(self.literales | other.literales)

    def __sub__(self, other):
        return Clausula(self.literales - other.literales)

    def __repr__(self):
        return " OR ".join(map(str, self.literales))


class Resolucion:
    def __init__(self):
        self.clausulas = []
        self.vacio = Clausula(set())

    def agregar_clausula(self, clausula):
        self.clausulas.append(Clausula(clausula))

    def mostrar_clausulas(self):
        for clausula in self.clausulas:
            print(clausula)

    def negar_conjetura(self, conjetura):
        negada = [Clausula({-literal for literal in clausula}) for clausula in conjetura]
        self.clausulas = negada + self.clausulas

    def unificar_variables(self, conjetura):
        variables = {}
        letras = "abcdefghijklmnopqrstuvwxyz"
        for i, arg in enumerate(conjetura):
            variables[letras[i]] = arg
        return variables

    def sustituir_variables(self, variables):
        self.clausulas = [
            Clausula(
                {Literal(lit.predicado, tuple(variables.get(arg, arg) for arg in lit.argumentos)) for lit in clausula.literales}
            )
            for clausula in self.clausulas
        ]

    def encontrar_clausula_opuesta(self):
        primera_clausula = self.clausulas[0]
        for clausula in self.clausulas[1:]:
            for literal in clausula.literales:
                if -literal in primera_clausula.literales:
                    return clausula, literal
        return None, None

    def resolver_clausulas(self):
        while True:
            primera_clausula = self.clausulas[0]
            segunda_clausula, literal_opuesto = self.encontrar_clausula_opuesta()

            if not segunda_clausula:
                print("No se encontró una cláusula opuesta. Fin del proceso. Conjetura Falsa")
                return False

            resolvente = (primera_clausula | segunda_clausula) - Clausula({literal_opuesto, -literal_opuesto})

            if resolvente.literales == self.vacio.literales:
                print("Se ha derivado la cláusula vacía. Contradicción encontrada. La conjetura es verdadera")
                return True

            self.clausulas.insert(0, resolvente)
            self.clausulas.remove(primera_clausula)
            self.clausulas.remove(segunda_clausula)
            print("Nueva cláusula generada:", resolvente)


class Main:
    @staticmethod
    def ejecutar():
        kb = Resolucion()
        kb.agregar_clausula({Literal("Hombre", ("a",))})
        kb.agregar_clausula({Literal("Pompeyano", ("Marco",))})
        kb.agregar_clausula({Literal("-Pompeyano", ("a",)), Literal("Romano", ("a",))})
        kb.agregar_clausula({Literal("Gobernante", ("Cesar",))})
        kb.agregar_clausula({Literal("-Romano", ("a",)), Literal("Leal", ("a", "Cesar")), Literal("Odia", ("a", "Cesar"))})
        kb.agregar_clausula({Literal("-Hombre", ("a",)), Literal("-Gobernante", ("b",)), Literal("-IntentaAsesinar", ("a", "b")), Literal("-Leal", ("a", "b"))})
        kb.agregar_clausula({Literal("IntentaAsesinar", ("Marco", "Cesar"))})

        print("Base de conocimiento inicial:")
        kb.mostrar_clausulas()
        print("--------------------------")

        conjetura = [{Literal("Odia", ("Marco", "Cesar"))}]
        kb.negar_conjetura(conjetura)

        variables = kb.unificar_variables(["Marco", "Cesar"])
        kb.sustituir_variables(variables)
        print("Después de sustituir variables:")
        kb.mostrar_clausulas()
        print("--------------------------")

        kb.resolver_clausulas()


if __name__ == "__main__":
    Main.ejecutar()

: 